---

## **Modelo de Machine Learning** para predecir la ESPERANZA DE VIDA en un país determinado a partir de diferentes indicadores asociados a economía, educación, salud y desarrollo social.

---

Comenzamos a trabajar en el desarrollo del modelo de machine learning. 
En una primera parte realizaremos el análisis exploratorio de los datos para quedarnos con un dataset limpio y ordenado, seleccionando aquellas variables que nos permita realizar un modelado adecuado. 
Luego procederemos a entrenar el modelo y por último realizaremos las evaluaciones pertinentes sobre el resultado de nuestro modelo.

## EDA

In [ ]:
# importamos algunas de las librerías a utilizar
import numpy as np
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Leemos el archivo

Leemos el archivo que vamos a trabajar, el mismo proviene del proceso de ETL realizado previamente y refiere a la tabla de Hechos definida en nuestra base de datos

In [ ]:
data = pd.read_csv('facts.csv')

In [ ]:
data

### Revisión de nulos

Observamos la cantidad de nulos que tenemos por cada variable

In [ ]:
data_null = data[data['value'].isna()]
series = data_null['series_id'].unique()
countries = data_null['country_id'].unique()
naSummary = pd.DataFrame(columns=['country_id','series','#NaN'])
for c in countries:
    df_facts_null_c = data_null[data_null['country_id']==c]
    for s in series:
        df_facts_null_cs = df_facts_null_c[df_facts_null_c['series_id']==s]
        l = sum(df_facts_null_cs['value'].isna())
        if l > 0:
            naSummary = pd.concat([naSummary,pd.DataFrame([[c,s,l]],columns=['country_id','series','#NaN'])],ignore_index=True)

naSummary

In [ ]:
#total de nulos
naSummary['#NaN'].sum()

Agrupamos los datos nulos por variable y pais

In [ ]:
naSummary.groupby('series').sum().sort_values('#NaN')

Se observa que 19 de las 20 variables con las que contamos contienen valores nulos

Nos quedamos con las variables que tienen menos de 30 nulos por pais

In [ ]:
naSummaryLess10 = naSummary[naSummary['#NaN']<30]
naSummaryLess10

In [ ]:
#total de nulos actuales
naSummaryLess10['#NaN'].sum()

### Imputación de valores nulos

En esta sección procederemos a imputar los valores nulos utilizando el modelo ARIMA (AutoRegressive Integrated Moving Average) el mismo es un modelo estadístico ampliamente utilizado para analizar y predecir series temporales.

Como contamos con un dataset con relativamente pocos años de análisis, consideramos aumentar la cantidad de años para combinarlo con el dataset utilizado hasta aqui y lograr una mejor performance del modelo ARIMA y así obtener mejores predicciones para nuestras imputaciones. Para esto se creo una tabla en la sección de ETL con datos desde el año 1960 a utilizar a continuación.

In [ ]:
data_60 = pd.read_csv('facts_1960.csv')
data_60.head()

In [ ]:
print('cantidad de nulos previamente:', data_60.value.isna().sum())

Procedemos a trabajar con el modelo ARIMA

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
for i in range(len(naSummaryLess10)):
    c = naSummaryLess10.iloc[i,0]
    s = naSummaryLess10.iloc[i,1]
    df_facts_cs = data_60[(data_60['country_id']==c)&(data_60['series_id']==s)]

    #print(df_facts_cs)
    
    y = df_facts_cs['value'].values
    ARIMAmodel = ARIMA(y, order = (1, 0, 1))
    ARIMAmodel = ARIMAmodel.fit()

    indexNaN = df_facts_cs[df_facts_cs['value'].isna()].index
    for j in indexNaN:
        year = int(df_facts_cs.loc[j,'year'])
        if year >= 1993:
            x = year - 1960
            y_pred = ARIMAmodel.predict(x)
            #print('Imputation on country: ',c,', series: ',s,', year: ',str(year),' and value: ',y_pred[0])

            df_facts_cs.loc[j,'value'] = y_pred[0]

    data_60[(data_60['country_id']==c)&(data_60['series_id']==s)] = df_facts_cs

    

In [ ]:
print('cantidad de nulos posteriormente:', data_60.value.isna().sum())

In [ ]:
indicators = ["GC.XPN.TOTL.GD.ZS", "NE.RSB.GNFS.CD", "NY.GDP.MKTP.KD.ZG", "NY.GDP.PCAP.CD",
                "NY.GNP.PCAP.CD", "SE.XPD.TOTL.GD.ZS", "SE.XPD.PRIM.PC.ZS", "SE.ADT.LITR.ZS",
                "SH.XPD.GHED.GD.ZS", "SP.DYN.LE00.IN", "SH.DYN.NMRT", "SH.DYN.MORT",
                "SN.ITK.DEFC.ZS", "IP.PAT.RESD", "GB.XPD.RSDV.GD.ZS", "SP.POP.SCIE.RD.P6",
                "SI.POV.GINI", "VC.IHR.PSRC.P5", "SI.POV.NAHC", "SL.UEM.TOTL.ZS"]

Chequeamos los nulos resultantes por variable y pais 

In [ ]:
data_30 = data_60[data_60['year'].astype(int)>=1993]
data_null = data_30[data_30['value'].isna()]
series = data_null['series_id'].unique()
countries = data_null['country_id'].unique()
naSummary = pd.DataFrame(columns=['country_id','series','#NaN'])
for c in countries:
    data_null_c = data_null[data_null['country_id']==c]
    for s in series:
        data_null_cs = data_null_c[data_null_c['series_id']==s]
        l = sum(data_null_cs['value'].isna())
        if l > 0:
            naSummary = pd.concat([naSummary,pd.DataFrame([[c,s,l]],columns=['country_id','series','#NaN'])],ignore_index=True)

for i in indicators:
    if i not in naSummary['series'].values:
        naSummary = pd.concat([naSummary,pd.DataFrame([['',i,0]],columns=['country_id','series','#NaN'])],ignore_index=True)

naSummary

In [ ]:
#total de nulos luego de imputar
naSummary['#NaN'].sum()

Observamos cuales variables nos han quedado sin valores nulos 

In [ ]:
naSummary.groupby('series').sum().sort_values('#NaN')

Extraemos las series sin nulos

In [ ]:
series_sin_nulos = naSummary[naSummary['#NaN'] == 0]['series'].tolist()

# Muestra las series sin nulos
series_sin_nulos

Comparamos graficamente los datos con y sin imputación para observar los cambios

In [ ]:
data_60_NaN = pd.read_csv('facts_1960.csv')

In [ ]:
import matplotlib.pyplot as plt

# Filtrar los datos para la serie y país específicos
serie = 'GC.XPN.TOTL.GD.ZS'
pais = 'ARG'
data_serie = data_60[(data_60['series_id'] == serie) & (data_60['country_id'] == pais)]
data_serie_nan = data_60_NaN[(data_60_NaN['series_id'] == serie) & (data_60_NaN['country_id'] == pais)]

# Crear el gráfico
plt.figure(figsize=(10, 6))
plt.plot(data_serie['year'], data_serie['value'], label='data_60')
plt.plot(data_serie_nan['year'], data_serie_nan['value'], label='data_60_NaN', linestyle='dashed', color='red')

# Agregar etiquetas y leyenda
plt.xlabel('Year')
plt.ylabel('Value')
plt.title(f'Comparación de la serie {serie} para {pais}')
plt.legend()

# Mostrar el gráfico
plt.show()

Como se pudo observar, de las 19 variables que contenían valores nulos antes de las imputaciones, sólo nos han quedado 8. Esto nos permite llevar la cantidad de variables aptas para consumir por el modelo de predicción de 1 a 12. Consideramos que es una cantidad de datos adecuada para obtener resultados satisfactorios.

### Revisión de duplicados

In [ ]:
data.duplicated().sum()

## Modelado

### Reconfiguración del dataset

Como se observa, la estructura del dataframe no es el más adecuado para el modelado que queremos realizar, por lo tanto procedemos a darle una estructura más óptima

In [ ]:
data_30

In [ ]:
# Filtrar las filas que corresponden a las series sin nulos
data_filtrado = data_30[data_30['series_id'].isin(series_sin_nulos)]

# Crear una tabla pivote
data = data_filtrado.pivot_table(index=['country_id', 'year'], columns='series_id', values='value').reset_index()

# Mostrar el DataFrame resultante
data.head()

Cambiamos los nombres a las variables para identificarlas 

In [ ]:
nombres = {
    'GC.XPN.TOTL.GD.ZS': 'Gasto(%_PIB)',
    'NE.RSB.GNFS.CD': 'Bal_comercial(US$)',
    'NY.GDP.MKTP.KD.ZG': 'PIB_crec(%_anual)',
    'NY.GDP.PCAP.CD': 'PIB_per_cap(US$)',
    'NY.GNP.PCAP.CD':'INB_per_cap(US$)',
    'SE.XPD.TOTL.GD.ZS':'Educacion(%_PIB)',
    'SE.XPD.PRIM.PC.ZS':'Gasto_alumno_primaria(%_PIB_per_capita)',
    'SE.ADT.LITR.ZS':'Tasa_alfabetizacion_adultos(%15_años_o_mas)',
    'SH.XPD.GHED.GD.ZS':'Salud(%_PIB)',
    'SP.DYN.LE00.IN':'EDV',
    'SH.DYN.NMRT':'Mortalidad_neo',
    'SH.DYN.MORT':'Mortalidad_5',
    'SN.ITK.DEFC.ZS':'Desnutricion(%_poblacion)',
    'IP.PAT.RESD':'Patentes',
    'GB.XPD.RSDV.GD.ZS':'I+D(%_PIB)',
    'SP.POP.SCIE.RD.P6':'Investigadores',
    'SI.POV.GINI':'Gini',
    'VC.IHR.PSRC.P5':'Homicidios',
    'SI.POV.NAHC':'Pobreza',
    'SL.UEM.TOTL.ZS':'Desempleo',
    'year':'Año',
    'country_id':'Pais_id'
}

In [ ]:
# Renombrar las columnas utilizando el método rename
data.rename(columns=nombres, inplace=True)

In [ ]:
data

In [ ]:
data.info()

Exportamos la tabla resultante

In [ ]:
data.to_csv('tabla_ML.csv',index=False)

### Análisis de correlación

#### **Gráfico general**

In [ ]:
corr = data.drop(['Pais_id'], axis=1).corr()
corr.style.background_gradient(cmap='coolwarm')

#### **Correlación de la variable objetivo 'EDV' con las variables predictoras:**


In [ ]:
correlation_matrix = corr.corr()
correlation_with_edv = correlation_matrix['EDV'].sort_values(ascending=False)
print("Correlación con 'EDV':")
print(correlation_with_edv)

**Interpretación de la correlación con 'EDV':**

- Se observa una correlación positiva fuerte con variables como 'PIB_per_cap(US$)', 'INB_per_cap(US$)', 'Salud(%_PIB)', y 'Año'. Esto sugiere que a medida que estas variables aumentan, la esperanza de vida ('EDV') tiende a aumentar.
- Hay correlaciones negativas fuertes con 'Homicidios', 'Mortalidad_neo', y 'Mortalidad_5'. Esto indica que a medida que estas variables aumentan, la esperanza de vida tiende a disminuir.
- Ambos análisis son a piori lógicos.

#### **Correlacion entre variables independientes:**

**Se observa una alta correlación entre dos pares de variables, lo cual conlleva riesgo de MULTICOLINEALIDAD.**

- Respecto a las variables asociadas a las tasas de mortalidad en niños y neonatales se decide quedarse con la primera ya que esta incluye a la otra. 
- En el caso de las variables asociadas al PBI la interpretación conceptual sugiere que miden aspectos diferentes del PIB, podría tener sentido incluir ambas variables en el modelo de regresión lineal ya que podrían ser importantes para explicar la variabilidad en la EDV. Por ello se realiza un análisis más profundo antes de tomar una decisión.

In [ ]:
# Evaluamos la correlación simple entre ambas variables
correlation = data[['PIB_crec(%_anual)', 'PIB_per_cap(US$)']].corr()
correlation

En base a los resultados de la correlación simple entre las variables "PIB_crec(%_anual)" y "PIB_per_cap(US$)", observamos que la correlación es relativamente baja, con un valor de aproximadamente -0.106. Este valor cercano a cero indica una correlación débil entre las dos variables.

La baja correlación sugiere que estas dos variables no están fuertemente relacionadas linealmente. En este caso, es menos probable que la multicolinealidad sea un problema significativo entre estas dos variables. 

Sin embargo, para obtener una evaluación más completa de la multicolinealidad, se procede a calcular los factores de inflación de la varianza (VIF)
para confirmar la ausencia de multicolinealidad significativa.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

variables = data[['PIB_crec(%_anual)', 'PIB_per_cap(US$)']]

# Agrega una constante para calcular el VIF
variables['constante'] = 1

# Calcula el VIF para cada variable
vif_data = pd.DataFrame()
vif_data["Variable"] = variables.columns
vif_data["VIF"] = [variance_inflation_factor(variables.values, i) for i in range(variables.shape[1])]

vif_data

Los resultados de los Factores de Inflación de la Varianza (VIF) muestran que los valores son relativamente bajos para ambas variables, "PIB_crec(%_anual)" y "PIB_per_cap(US$)", con valores de alrededor de 1.01. En general, VIF cercanos a 1 indican que no hay una alta multicolinealidad entre las variables.

Además, el VIF de la constante es de aproximadamente 2.85, lo cual es bastante bajo. El VIF de la constante indica cuánto se infla la varianza de los coeficientes debido a la multicolinealidad, y valores bajos son deseables.

Dado que los VIF son bajos en este caso, no parece haber una multicolinealidad significativa entre las variables "PIB_crec(%_anual)" y "PIB_per_cap(US$)".

### Definimos la variable a predecir (Y) y las variables predictoras (X)

In [ ]:
y = data["EDV"]
x = data.drop(['EDV','Pais_id','Mortalidad_neo'], axis = 1)

In [ ]:
pd.DataFrame(y)

In [ ]:
x

### Detección de outliers

Se analiza las variables en búsqueda de posibles valores atípicos que puedan generar inconvenientes en nuestros resultados. Observamos en un primer análisis gráficos de caja, los cuales nos ofrecen una rápida muestra visual de la distribución de los valores de cada variable.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Número de variables
num_variables = len(x.columns)

# Número de filas y columnas para organizar los gráficos
num_rows = num_variables // 3 + (num_variables % 3 > 0)
num_cols = 3

# Crea subgráficos
fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 4 * num_rows))
fig.suptitle('Boxplots de Variables Predictoras', y=1.02)

# Itera sobre las variables y crea los boxplots
for i, (variable, ax) in enumerate(zip(x.columns, axes.flatten())):
    sns.boxplot(x=x[variable], ax=ax)
    ax.set_title(f'{variable}')

# Ajusta el diseño para evitar solapamiento
plt.tight_layout()

# Muestra la visualización
plt.show()

**Si bien se observan en la mayoría de las variables algunos valores alejados del rango intercuartílico, esto puede tener sentido y no se sería apropiado en un primer análisis descartarlos. Hay que recordar que nuestro dataset se compone de paises muy variados en todos los términos de los factores observados por lo cual es esperable encontrar una amplia variabilidad en los datos debido a las diferencias naturales entre las entidades estudiadas, esto además puedo enriquecer a nuestro modelo.**

Sin embargo, antes de quedarnos completamente con esa conclusión, analizaremos con mayor detalle algunas variables.

Educación:

**Análisis Estadístico:**

Utilizamos medidas estadísticas como el rango intercuartílico (IQR) para identificar outliers.
Definimos umbrales para identificar valores atípicos. Los valores que están por debajo de Q1 - 1.5 * IQR o por encima de Q3 + 1.5 * IQR podrían considerarse outliers.

In [ ]:
# Calcula el rango intercuartílico (IQR) para 'Educacion(%_PIB)'
Q1_educacion = x['Educacion(%_PIB)'].quantile(0.25)
Q3_educacion = x['Educacion(%_PIB)'].quantile(0.75)
IQR_educacion = Q3_educacion - Q1_educacion

# Define umbrales para identificar outliers en 'Educacion(%_PIB)'
lower_bound_educacion = Q1_educacion - 1.5 * IQR_educacion
upper_bound_educacion = Q3_educacion + 1.5 * IQR_educacion

# Identifica outliers en 'Educacion(%_PIB)'
outliers_educacion = (x['Educacion(%_PIB)'] < lower_bound_educacion) | (x['Educacion(%_PIB)'] > upper_bound_educacion)

# Filtra los outliers en 'x' y obtén los correspondientes datos de 'y' (esperanza de vida)
outliers_data = data[outliers_educacion]
outliers_paises_anios = outliers_data[['Pais_id', 'Año']]

# Muestra los registros atípicos y a qué país y años corresponden
print("Registros Atípicos en 'Educacion(%_PIB)':")
outliers_data[['Pais_id', 'Año','Educacion(%_PIB)']]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Crea un boxplot de la variable 'Educacion' después de Winsorizing
plt.figure(figsize=(8, 6))
sns.boxplot(x['Educacion(%_PIB)'])
plt.title('Boxplot de la Variable "Educacion(%_PIB)" antes de Winsorizing')
plt.show()

In [ ]:
from scipy.stats.mstats import winsorize

# Define los límites de Winsorizing (ajusta según tu preferencia)
lower_limit = 0.05
upper_limit = 0.05

# Aplica Winsorizing a la variable 'Educacion'
x['Educacion(%_PIB)'] = winsorize(x['Educacion(%_PIB)'], limits=[lower_limit, upper_limit])

# Puedes ajustar los límites según tu necesidad

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Crea un boxplot de la variable 'Educacion' después de Winsorizing
plt.figure(figsize=(8, 6))
sns.boxplot(x['Educacion(%_PIB)'])
plt.title('Boxplot de la Variable "Educacion(%_PIB)" después de Winsorizing')
plt.show()

En el caso de la variable Educación, se puede observar que la gran mayoría de los datos por encima del máximo del boxplot corresponden a Cuba, indicando que el país ha invertido en Educación mucho más que el resto de los paises observados. Descartarndo estos valores perderíamos esa información que puede resultar valiosa para el modelo.

Homicidios

In [ ]:
# Calcula el rango intercuartílico (IQR) para 'Homicidios'
Q1_homicidios = x['Homicidios'].quantile(0.25)
Q3_homicidios = x['Homicidios'].quantile(0.75)
IQR_homicidios = Q3_homicidios - Q1_homicidios

# Define umbrales para identificar outliers en 'Homicidios'
lower_bound_homicidios = Q1_homicidios - 1.5 * IQR_homicidios
upper_bound_homicidios = Q3_homicidios + 1.5 * IQR_homicidios

# Identifica outliers en 'Homicidios'
outliers_homicidios = (x['Homicidios'] < lower_bound_homicidios) | (x['Homicidios'] > upper_bound_homicidios)

# Filtra los outliers en 'x' y obtén los correspondientes datos de 'y' (esperanza de vida)
outliers_data_homicidios = data[outliers_homicidios]
outliers_paises_anios_homicidios = outliers_data_homicidios[['Pais_id', 'Año']]

# Muestra los registros atípicos y a qué país y años corresponden
print("Registros Atípicos en 'Homicidios':")
outliers_data_homicidios[['Pais_id', 'Año','Homicidios']].sample(40, random_state=126).sort_values(by='Homicidios', ascending=False)

Se analizaron los outliers en la variable 'Homicidios' y al igual que eduación, se observa que los valores si bien estan alejados del rango intercuartílico se explica desde la naturaleza de las observaciones. En este caso la mayoría de los valores por encima del maximo corresponde a El Salvador, un país que se conoce por una alta tasa de homicidios intencionados.

**Con esto reafirmamos la desición de mantener todos los valores**

### Split en train y test

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, random_state =123)

In [ ]:
print('Tamaño del set de train',x_train.shape)
print('Tamaño del set de test:',x_test.shape)

### Normalización de los datos

In [ ]:
# Observamos los valores estadísticos previo al escalamiento
data.describe()

In [ ]:
# Utilizamos la librería de sklearn para escalar nuestras variables predictoras
from sklearn.preprocessing import StandardScaler

esc = StandardScaler()

x_train_esc = esc.fit_transform(x_train)
x_test_esc = esc.transform(x_test)

### Entrenamiento del modelo

In [ ]:
# Entrenamos el modelo utilizando la librería sklearn
from sklearn.linear_model import LinearRegression

modelo = LinearRegression()
modelo.fit(x_train_esc, y_train)

### Predicción

In [ ]:
# Definimos la variable de predicción 
y_pred = modelo.predict(x_test_esc)

In [ ]:
y_pred.size

### Validaciones del Modelo de Regresión:
Se procede a evaluar si se  con las suposiciones del modelo utilizado

- *Linealidad:* entre las variables independientes y la variable dependiente. Utilizamos gráfico de dispersión.

- *Independencia:* Asegurar que los errores (residuos) no estén correlacionados entre sí. Examinamos el gráfico de residuos.

- *Homocedasticidad:* Verificar que la varianza de los errores sea constante en todos los niveles de las variables independientes. Lo observamos en un gráfico de residuos frente a las predicciones.

- *Normalidad de los Residuos:* Comprobar que los residuos sigan una distribución normal. Lo evaluamos mediante un histograma de los residuos.

Linealidad (Gráfico de Dispersión):

- Un gráfico simple donde el eje x representa los valores reales y el eje y representa las predicciones.
- Vemos cómo se alinean las predicciones con los valores reales. 
- Si la relación entre las variables es lineal, deberíamos ver una dispersión uniforme alrededor de la línea diagonal.

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(y_test, y_pred)
plt.xlabel('Valores Reales')
plt.ylabel('Predicciones')
plt.title('Gráfico de Dispersión: Valores Reales vs. Predicciones')
plt.show()

Independencia (Gráfico de Residuos):

- Un gráfico que muestra la diferencia entre los valores reales y las predicciones en función de los valores reales.
- Podemos identificar patrones en los residuos y verificar si hay heterocedasticidad.
- Deberían distribuirse aleatoriamente alrededor de la línea horizontal cero. No debería haber un patrón discernible.

In [ ]:
residuals = y_test - y_pred
plt.scatter(y_test, residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Valores Reales')
plt.ylabel('Residuos')
plt.title('Gráfico de Residuos')
plt.show()


Homocedasticidad (Gráfico de Residuos vs. Predicciones):

- Aquí, buscamos una dispersión constante de los residuos en todos los niveles de las predicciones. 
- No debería haber un patrón en forma de cono o embudo.

In [ ]:
plt.scatter(y_pred, residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicciones (y_pred)')
plt.ylabel('Residuos')
plt.title('Gráfico de Residuos vs. Predicciones')
plt.show()

Histograma de residuos:

- Un histograma que muestra la distribución de los residuos.
- Puedes verificar si los residuos siguen una distribución normal.

In [ ]:
plt.hist(residuals, bins=30)
plt.xlabel('Residuos')
plt.ylabel('Frecuencia')
plt.title('Histograma de Residuos')
plt.show()

**Se pudo validar todas las suposiciones del modelo de regresión lineal, por lo tanto se procede a evaluar las métricas de los resultados obtenido en el modelo**

### Métricas de desempeño

#### **Error Absoluto Medio (MAE):**

- El MAE mide el promedio de las diferencias absolutas entre las predicciones y los valores reales.
- Cuanto más bajo sea el MAE, mejor será el rendimiento.

In [ ]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_test, y_pred)
print(f'MAE: {mae}')

In [ ]:
import matplotlib.pyplot as plt

# Calcula los errores absolutos
mae_errors = np.abs(y_test - y_pred)

# Gráfico de dispersión de MAE
plt.scatter(y_test, mae_errors)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Valores Reales (y_test)')
plt.ylabel('Errores Absolutos')
plt.title('Gráfico de Dispersión: Valores Reales vs. Errores Absolutos (MAE)')
plt.show()

**El MAE (Error Absoluto Medio) de 1.1977 significa que, en promedio, las predicciones de esperanza de vida difieren en aproximadamente 1.2 años de los valores reales**

Si bien el resultado ya parece ser muy bueno, para obtener una evaluación más efectiva de este resultado la llevaremos a nuestro contexto, haciendo una comparación con la magnitud de la EDV en nuestro conjunto de datos:

In [ ]:
mean_life_expectancy = y_test.mean()
std_life_expectancy = y_test.std()

print(f'Media de Esperanza de Vida: {mean_life_expectancy:.2f} años')
print(f'Desviación Estándar de Esperanza de Vida: {std_life_expectancy:.2f} años')

mae_percentage_of_mean = (mae / mean_life_expectancy) * 100

print(f'MAE como porcentaje de la Media: {mae_percentage_of_mean:.2f}%')

**Interpretación:**

- El MAE del 1.57% sugiere que, en promedio, las predicciones son bastante precisas en relación con la magnitud de la esperanza de vida en tu conjunto de datos.
- En el contexto de la esperanza de vida, un MAE del 1.55% podría considerarse como un buen desempeño, especialmente si la variabilidad natural de la esperanza de vida en nuestro conjunto de datos es mayor que este porcentaje.

#### **Error Cuadrático Medio (MSE):**

- El MSE mide el promedio de los cuadrados de las diferencias entre las predicciones y los valores reales.
- Penaliza errores más grandes más fuertemente que el MAE.

In [ ]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, y_pred)
print(f'MSE: {mse}')

Tal como el valor del MAE, el MSE es realtivamente pequeño respecto a la media de EVD, afirma el buen desempeño de nuestro modelo de predicción

**RMSE (Raíz del Error Cuadrático Medio):**

- El RMSE es simplemente la raíz cuadrada del MSE.
- Es más interpretable que el MSE ya que proporciona una medida del error en la misma escala que la variable dependiente. Cuanto más cercano a cero, mejor.

In [ ]:
rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

#### **Coeficiente de Determinación (R²):**

- El R² indica la proporción de la variabilidad en la variable dependiente que es predecible a partir de las variables independientes.
- Un valor más cercano a 1 indica un mejor ajuste.

In [ ]:
from sklearn.metrics import r2_score

r2 = r2_score(y_test, y_pred)
print(f'R²: {r2}')

Un (R²) de 0.8977 significa que aproximadamente el 89.77% de la variabilidad en la esperanza de vida es explicada por las variables independientes incluidas en el modelo.

- **Interpretación:** Un (R²) de 0.8977 es bastante alto y sugiere que el modelo está capturando bien las tendencias y patrones en los datos de esperanza de vida. Casi el 90% de la variabilidad en la esperanza de vida se puede explicar mediante las variables incluidas en el modelo.

**CONCLUSIÓN: se evaluó el desempeño del modelo a través de distintas métricas y todas arrojan resultados positivos, indicando que nuestro modelo consigue realizar muy buenas predicciones del valor de esperanza de vida esperada.**

### Validación del modelo

**Validación Cruzada (Cross-Validation):**
- La validación cruzada k-fold divide el conjunto de datos en k partes (folds) y realiza k iteraciones de entrenamiento y evaluación. En cada iteración, un fold se utiliza como conjunto de prueba y los k-1 folds restantes se utilizan para entrenar el modelo. Esto proporciona k estimaciones de rendimiento que se promedian para obtener una medida general.

Creamos una función que utiliza la validación cruzada y obtiene las métricas de nuestro interés en cada iteración de la validación según el parámetro 'cv'. 

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

def cross_val_metrics(estimator, X, y, cv=5):
    # Realiza la validación cruzada
    y_pred = cross_val_predict(estimator, X, y, cv=cv)

    # Calcula las métricas para cada iteración
    metrics = []
    for i in range(cv):
        start = i * len(y) // cv
        end = (i + 1) * len(y) // cv
        y_true_i = y[start:end]
        y_pred_i = y_pred[start:end]

        mae_i = mean_absolute_error(y_true_i, y_pred_i)
        mse_i = mean_squared_error(y_true_i, y_pred_i)
        rmse_i = np.sqrt(mse_i)
        r2_i = r2_score(y_true_i, y_pred_i)

        metrics.append({'MAE': mae_i, 'MSE': mse_i, 'RMSE': rmse_i, 'R²': r2_i})

    return metrics

In [ ]:
# Uso de la función con nuestro modelo
metrics_per_iteration = cross_val_metrics(modelo, x, y, cv=5)

# Imprime los resultados para cada iteración
for i, metrics in enumerate(metrics_per_iteration, 1):
    print(f'Iteración {i}: {metrics}')

# Calcula el promedio de cada métrica
avg_metrics = {
    'MAE': np.mean([m['MAE'] for m in metrics_per_iteration]),
    'MSE': np.mean([m['MSE'] for m in metrics_per_iteration]),
    'RMSE': np.mean([m['RMSE'] for m in metrics_per_iteration]),
    'R²': np.mean([m['R²'] for m in metrics_per_iteration]),
}

print(f'Promedio de métricas: {avg_metrics}')

Se observa que si bien hay cierta variabilidad en los resultados siguen siendo buenos resultados. De esta forma validamos el desempeño del modelo. 

### Interpretabilidad del modelo

In [ ]:
print("Coeficientes:")
for feature, coef in zip(x.columns, modelo.coef_):
    print(f"{feature}: {coef}")

print(f"Intercepto: {modelo.intercept_}")

---

### Modelo sin variable 'Año'

In [ ]:
y2 = data["EDV"]
x2 = data.drop(['EDV','Pais_id','Mortalidad_neo','Año'], axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split

x_train2, x_test2, y_train2, y_test2 = train_test_split(x2,y2, test_size = 0.3, random_state =123)

In [ ]:
from sklearn.preprocessing import StandardScaler

esc = StandardScaler()

x_train2_esc = esc.fit_transform(x_train2)
x_test2_esc = esc.transform(x_test2)

In [ ]:
from sklearn.linear_model import LinearRegression

modelo2 = LinearRegression()
modelo2.fit(x_train2_esc, y_train2)

In [ ]:
y_pred2 = modelo2.predict(x_test2_esc)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae2 = mean_absolute_error(y_test2, y_pred2)
mse2 = mean_squared_error(y_test2, y_pred2)
rmse2 = np.sqrt(mse2)
r22 = r2_score(y_test2, y_pred2)

print(f'MAE2: {mae2}')
print(f'MSE2: {mse2}')
print(f'RMSE2: {rmse2}')
print(f'R²2: {r22}')

In [ ]:
# Uso de la función con nuestro modelo
metrics_per_iteration = cross_val_metrics(modelo2, x2, y2, cv=5)

# Imprime los resultados para cada iteración
for i, metrics in enumerate(metrics_per_iteration, 1):
    print(f'Iteración {i}: {metrics}')

# Calcula el promedio de cada métrica
avg_metrics = {
    'MAE': np.mean([m['MAE'] for m in metrics_per_iteration]),
    'MSE': np.mean([m['MSE'] for m in metrics_per_iteration]),
    'RMSE': np.mean([m['RMSE'] for m in metrics_per_iteration]),
    'R²': np.mean([m['R²'] for m in metrics_per_iteration]),
}

print(f'Promedio de métricas: {avg_metrics}')

In [ ]:
print("Coeficientes:")
for feature, coef in zip(x2.columns, modelo.coef_):
    print(f"{feature}: {coef}")

print(f"Intercepto: {modelo.intercept_}")

---

### Testeo con nuevos datos

Probamos nuestro modelo en un entorno como en el que se desempeñará

In [ ]:
# Tomamos como ejemplo una muestra de una fila de nuestros datos
n_filas_muestra = 1  
x_new = x.sample(n=n_filas_muestra, random_state=126)

In [ ]:
data.iloc[x_new.index.values]['EDV'] #traemos el valor de EDV para comparar

Creamos una función que utiliza nuestro modelo y toma nuevos datos para predecir a partir de estos una EDV esperada

In [ ]:
from sklearn.preprocessing import StandardScaler

# Ajusta el scaler con los datos de entrenamiento
esc = StandardScaler()
esc.fit(x_train)

def predecir_esperanza_vida(modelo, scaler, x_train, nuevos_datos):
    # Asegura que los nuevos datos tengan los mismos nombres de columnas que los datos de entrenamiento
    nuevos_datos.columns = x_train.columns
    
    # Escala los nuevos datos usando el scaler ajustado con los datos de entrenamiento
    nuevos_datos_scaled = scaler.transform(nuevos_datos)
    
    # Realiza la predicción
    prediccion = modelo.predict(nuevos_datos_scaled)
    
    # Imprime el resultado
    print(f"La esperanza de vida predicha es: {round(prediccion.item(),2)} años")

In [ ]:
# Llama a la función con los valores de las variables independientes
predecir_esperanza_vida(modelo, esc, x_train, x_new)

Función si los nuevos datos de entrena se componen con más de un valor por variable

In [ ]:
def predecir_esperanza_vida(modelo, scaler, x_train, nuevos_datos):
    # Asegúrate de que los nuevos datos tengan los mismos nombres de columnas que los datos de entrenamiento
    nuevos_datos.columns = x_train.columns
    
    # Si hay más de una fila en nuevos_datos, ajusta el scaler con los datos de entrenamiento
    if len(nuevos_datos) > 1:
        scaler.fit(x_train)
    
    # Escala los nuevos datos usando el scaler
    nuevos_datos_scaled = scaler.transform(nuevos_datos)
    
    # Realiza la predicción
    prediccion = modelo.predict(nuevos_datos_scaled)
    
    # Imprime el resultado
    if len(nuevos_datos) == 1:
        print(f"La esperanza de vida predicha es: {round(prediccion.item(), 2)} años")
    else:
        for i, pred in enumerate(prediccion):
            print(f"Para la fila {i + 1}, la esperanza de vida predicha es: {round(pred.item(), 2)} años")

### Preparamos y guardamos nuestro modelo en un archivo consumible por nuestro servidor 

In [ ]:
# creamos un pipeline para poder correr nuestro modelo en nuestro servidor
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(esc, modelo)

In [ ]:
# corroboramos los resultados
pipe.fit(x_train, y_train)
pipe.predict(x_new)

In [ ]:
# utilizamos la librería joblib para guardar nuestro pipeline
import joblib
joblib.dump(pipe, 'pipe.joblib')

In [ ]:
# load the pipeline from a file
same_pipe = joblib.load('pipe.joblib')

In [ ]:
same_pipe.predict(x_new)

---